# Basic setup.

In [9]:
#-Installing the required modules-#
%pip install pandas
%pip install elasticsearch


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
#-Importing the modules-#
import warnings
import elasticsearch

#-Suppressing the warnings-#
warnings.filterwarnings("ignore", category = elasticsearch.ElasticsearchWarning)

In [11]:
#-Creating the elastic search objec and testing its connection-#
host = [{"scheme": "http", "host": "localhost", "port": 9200}]
es = elasticsearch.Elasticsearch(hosts = host, request_timeout = 600)
es.ping()


True

# Index Operations.

In [12]:
#-Removing any existing instance of the requried indices-#
es.indices.delete(index = "testindex", ignore_unavailable = True)
es.indices.delete(index = "tbl_reporters", ignore_unavailable = True)

ObjectApiResponse({'acknowledged': True})

In [13]:
#-Creating an index called tbl_reporters-#
es.indices.create(index = "tbl_reporters")


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'tbl_reporters'})

In [14]:
#-Getting all the indices and checking if tbl_reporters is present-#
response = es.indices.get(index = "*", ignore_unavailable = True)
print("tbl_reporters" in response)


True


In [15]:
#-Deleting the index-#
es.indices.delete(index = "tbl_reporters", ignore_unavailable = True)


ObjectApiResponse({'acknowledged': True})

# Uploading sample JSON Docs.

In [16]:
#-Document dictionaries-#
doc1 = {
    "reporter_id": 1,
    "name": "Bobulagumon.",
    "age": 24,
    "active": True,
    "status": 1,
    "outlet": "Onclusive"
}
doc2 = {
    "reporter_id": 2,
    "name": "Veerbamba.",
    "age": 25,
    "active": False,
    "status": 2,
    "outlet": "Critical Mention"
}


In [17]:
#-Creating a temp index-#
es.indices.create(index = "testindex")


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'testindex'})

In [18]:
#-Adding the document to the newly created index-#
response = es.index(index = "testindex", body = doc1, id = 1)
print(response)
response = es.index(index = "testindex", body = doc2, id = 2)
print(response)


{'_index': 'testindex', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'testindex', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [19]:
#-Getting a document by ID-#
response = es.get(index = "testindex", id = 2)
print("Document by id.")
print(response, "\n")

#-Getting all the documents-#
response = es.search(index = "testindex")
print("All documents.")
for res in response["hits"]["hits"]:
    print(res)


Document by id.
{'_index': 'testindex', '_id': '2', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'reporter_id': 2, 'name': 'Veerbamba.', 'age': 25, 'active': False, 'status': 2, 'outlet': 'Critical Mention'}} 

All documents.


# Deleting a document.

In [20]:
#-Deleting a document by ID-#
es.delete(index = "testindex", id = 2)
print("Deleted successfully.")

#-Trying to fetch the document after deleting
try:
    es.get(index = "testindex", id = 2)
except Exception as e:
    print(e)


Deleted successfully.
NotFoundError(404, "{'_index': 'testindex', '_id': '2', 'found': False}")


# Additional index info.

In [23]:
#-Getting the index mappings-#
dict(es.indices.get_mapping(index = "testindex"))


{'testindex': {'mappings': {'properties': {'active': {'type': 'boolean'},
    'age': {'type': 'long'},
    'name': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'outlet': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'reporter_id': {'type': 'long'},
    'status': {'type': 'long'}}}}}

In [22]:
#-Getting the index settings-#
dict(es.indices.get_settings(index = "testindex"))


{'testindex': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'testindex',
    'creation_date': '1725872568761',
    'number_of_replicas': '1',
    'uuid': '1v46f7m0Q4WlwCuYmtGVHg',
    'version': {'created': '8512000'}}}}}

In [24]:
#-Getting the data using search-#
dict(es.search(index = "testindex"))


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'testindex',
    '_id': '1',
    '_score': 1.0,
    '_source': {'reporter_id': 1,
     'name': 'Bobulagumon.',
     'age': 24,
     'active': True,
     'status': 1,
     'outlet': 'Onclusive'}}]}}